In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ("samolyot","avtomobil","qush","mushuk","kiyik","it","qurbaqa","ot","kema","yuk mashinasi")

Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
from tqdm import tqdm

for epoch in range(10): 
    running_loss = 0.0
    print(f'Epoch:{epoch+1}',end="\t")
    for data in tqdm(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    print(f'Loss:{running_loss}')
    print()
    torch.save({
            'epoch': 10,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': running_loss,
            }, f'ckpt_torch/checkpoint{epoch+1}.pth.tar')
    #torch.save(net.state_dict(), f'ckpt_torch/checkpoint{epoch+1}.pth.tar')
    
print('Finished Training')

Epoch:1	

  0%|                                                                                          | 0/1563 [00:00<?, ?it/s]/home/mrfoziljon/checkpoint/env/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|███████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:16<00:00, 92.93it/s]


Loss:3493.059510707855

Epoch:2	

100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:13<00:00, 116.11it/s]


Loss:2876.055040001869

Epoch:3	

...

Loss:2009.733350098133

Epoch:8	

100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:13<00:00, 114.96it/s]


Loss:1925.6351432800293

Epoch:9	

 19%|██████████████▋                                                                | 290/1563 [00:02<00:11, 112.69it/s]


KeyboardInterrupt: 

In [8]:
checkpoint = torch.load('ckpt_torch/checkpoint8.pth.tar')

In [9]:
model = Net().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [10]:
model

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [11]:
for epoch in range(10): 
    running_loss = 0.0
    print(f'Epoch:{epoch+1}',end="\t")
    for data in tqdm(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    print(f'Loss:{running_loss}')
    print()

Epoch:1	

100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:14<00:00, 106.72it/s]


Loss:1850.8852034807205

Epoch:2	

100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:14<00:00, 109.95it/s]


Loss:1781.9004002809525

Epoch:3	

...

Loss:1502.5353444218636

Epoch:9	

100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:14<00:00, 110.08it/s]


Loss:1460.3194490075111

Epoch:10	

100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:13<00:00, 114.50it/s]

Loss:1426.8054774105549



In [12]:
torch.save({
            'epoch': 10,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': running_loss,
            }, f'ckpt_torch/model.pth.tar')